In [3]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.linkextractors import LinkExtractor
from scrapy.spiders import CrawlSpider, Rule

# pip install pytest-playwright
# playwright install
from playwright.sync_api import sync_playwright

import pandas as pd
from openpyxl import Workbook

#### Klasse Definieren
class CryptoSpider(CrawlSpider):
    name = 'crypto_spider'
    allowed_domains = ['crypto-fundraising.info']
    start_urls = ['https://crypto-fundraising.info/']
    rules = (
        Rule(LinkExtractor(restrict_css='table tr'), callback='parse_item', follow=True),
    )
    def parse_item(self, response):
        href = response.url
        yield {'href': href}
        # Use Playwright to fetch HTML data of the sub-links
        with sync_playwright() as playwright:
            browser = playwright.chromium.launch()
            page = browser.new_page()
            page.goto(href)
            html_content = page.content()
            browser.close()
        # Process the HTML data
        df_list = pd.read_html(html_content)  # Extract all tables from the HTML content
        print("Data:\n")
        print(df_list)
        # Iterate over each table and save them to separate sheets in an Excel file
        file_name = 'table_data.xlsx'
        with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
            for idx, df in enumerate(df_list, start=1):
                sheet_name = f'Table_{idx}'
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        self.log(f"Table data saved to {file_name}")
# Run the spider
process = CrawlerProcess()
process.crawl(CryptoSpider)
process.start()

2023-06-27 14:55:15 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: scrapybot)
2023-06-27 14:55:15 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.12, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.11.4 (tags/v3.11.4:d2340ef, Jun  7 2023, 05:45:37) [MSC v.1934 64 bit (AMD64)], pyOpenSSL 23.2.0 (OpenSSL 3.1.1 30 May 2023), cryptography 41.0.1, Platform Windows-10-10.0.19045-SP0
2023-06-27 14:55:15 [scrapy.crawler] INFO: Overridden settings:
{}
2023-06-27 14:55:15 [py.warnings] WARNING: C:\Users\Engineer\AppData\Roaming\Python\Python311\site-packages\scrapy\utils\request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

